# SPTAG python wrapper tutorial 

To end-to-end build a vector search online service, it contains two steps:
- Offline build SPTAG index for database vectors
- Online serve the index to support vector search requests from the clients

## Offline build SPTAG index

> Prepare input vectors and metadatas for SPTAG

In [18]:
import os
import shutil
import numpy as np

vector_number = 1000
vector_dimension = 100

# Randomly generate the database vectors. Currently SPTAG only support int8, int16 and float32 data type.
x = np.random.rand(vector_number, vector_dimension).astype(np.float32) 

# Prepare metadata for each vectors, separate them by '\n'. Currently SPTAG python wrapper only support '\n' as the separator
m = ''
for i in range(vector_number):
    m += str(i) + '\n'

> Build SPTAG index for database vectors **x**

In [3]:
import SPTAG

index = SPTAG.AnnIndex('BKT', 'Float', vector_dimension)

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("NumberOfThreads", '4', "Index")

# Set the distance type. Currently SPTAG only support Cosine and L2 distances. Here Cosine distance is not the Cosine similarity. The smaller Cosine distance it is, the better.
index.SetBuildParam("DistCalcMethod", 'L2', "Index") 

if (os.path.exists("sptag_index")):
    shutil.rmtree("sptag_index")
if index.BuildWithMetaData(x, m, vector_number, False, False):
    index.Save("sptag_index") # Save the index to the disk

os.listdir('sptag_index')

['deletes.bin',
 'graph.bin',
 'indexloader.ini',
 'metadata.bin',
 'metadataIndex.bin',
 'tree.bin',
 'vectors.bin']

In [4]:
# Local index test on the vector search
index = SPTAG.AnnIndex.Load('sptag_index')

# prepare query vector
q = np.random.rand(vector_dimension).astype(np.float32)

result = index.SearchWithMetaData(q, 10) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

[671, 27, 566, 905, 203, 277, 178, 28, 135, 845]
[10.656339645385742, 10.683115005493164, 10.721793174743652, 11.356607437133789, 11.375459671020508, 11.677772521972656, 11.731498718261719, 11.737524032592773, 11.911179542541504, 12.022208213806152]
[b'671\n', b'27\n', b'566\n', b'905\n', b'203\n', b'277\n', b'178\n', b'28\n', b'135\n', b'845\n']


> Build SPTAG index for database vector **x** with PQ quantization

In [21]:
import subprocess
import struct
import numpy as np

if (os.path.exists("quantizer.bin")):
    os.remove("quantizer.bin")
if (os.path.exists("quan_doc_vectors.bin")):
    os.remove("quan_doc_vectors.bin")
cmd = "Quantizer.exe -d %d -v Float -f DEFAULT -i sptag_index\\vectors.bin -o quan_doc_vectors.bin -oq quantizer.bin -qt PQQuantizer -qd %d -ts %d -norm false" % (vector_dimension, int(vector_dimension / 2), vector_number)
result = subprocess.run(cmd)
print (result) 
# For SPTAG index with quantization:
#     Cosine distance: norm -> true and use L2 for index build
#     L2 distance: norm -> false and use L2 for index build

f = open('quan_doc_vectors.bin', 'rb')
r = struct.unpack('i', f.read(4))[0]
c = struct.unpack('i', f.read(4))[0]
quan_x = np.frombuffer(f.read(r*c), dtype=np.uint8).reshape((r,c))
f.close()
print (quan_x.shape)

CompletedProcess(args='Quantizer.exe -d 100 -v Float -f DEFAULT -i sptag_index\\vectors.bin -o quan_doc_vectors.bin -oq quantizer.bin -qt PQQuantizer -qd 50 -ts 1000 -norm false', returncode=0)
(1000, 50)


In [6]:
import SPTAG

index = SPTAG.AnnIndex('BKT', 'UInt8', int(vector_dimension / 2))

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("NumberOfThreads", '4', "Index")

# Set the distance type. Currently SPTAG only support Cosine and L2 distances. Here Cosine distance is not the Cosine similarity. The smaller Cosine distance it is, the better.
index.SetBuildParam("DistCalcMethod", 'L2', "Index") 

if (os.path.exists("quan_sptag_index")):
    shutil.rmtree("quan_sptag_index")
if index.LoadQuantizer("quantizer.bin") and index.BuildWithMetaData(quan_x, m, vector_number, False, False):
    index.Save("quan_sptag_index") # Save the index to the disk

os.listdir('quan_sptag_index')

['deletes.bin',
 'graph.bin',
 'indexloader.ini',
 'metadata.bin',
 'metadataIndex.bin',
 'quantizer.bin',
 'tree.bin',
 'vectors.bin']

In [7]:
# Local index test on the vector search
index = SPTAG.AnnIndex.Load('quan_sptag_index')
index.SetQuantizerADC(True)
result = index.SearchWithMetaData(q, 12) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

[27, 671, 566, 905, 203, 277, 28, 178, 135, 845, 404, 54]
[10.690985679626465, 10.709616661071777, 10.901339530944824, 11.261568069458008, 11.372068405151367, 11.45018196105957, 11.7642183303833, 11.848787307739258, 12.02137565612793, 12.052336692810059, 12.079133033752441, 12.168403625488281]
[b'27\n', b'671\n', b'566\n', b'905\n', b'203\n', b'277\n', b'28\n', b'178\n', b'135\n', b'845\n', b'404\n', b'54\n']


In [19]:
import SPTAG

index = SPTAG.AnnIndex('SPANN', 'Float', vector_dimension)

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("IndexAlgoType", "BKT", "Base")
index.SetBuildParam("IndexDirectory", "spann_index", "Base")
index.SetBuildParam("DistCalcMethod", "L2", "Base")

index.SetBuildParam("isExecute", "true", "SelectHead")
index.SetBuildParam("NumberOfThreads", "4", "SelectHead")
index.SetBuildParam("Ratio", "0.2", "SelectHead") # index.SetBuildParam("Count", "200", "SelectHead")

index.SetBuildParam("isExecute", "true", "BuildHead")
index.SetBuildParam("RefineIterations", "3", "BuildHead")
index.SetBuildParam("NumberOfThreads", "4", "BuildHead")

index.SetBuildParam("isExecute", "true", "BuildSSDIndex")
index.SetBuildParam("BuildSsdIndex", "true", "BuildSSDIndex")
index.SetBuildParam("PostingPageLimit", "12", "BuildSSDIndex")
index.SetBuildParam("SearchPostingPageLimit", "12", "BuildSSDIndex")
index.SetBuildParam("NumberOfThreads", "4", "BuildSSDIndex")
index.SetBuildParam("InternalResultNum", "32", "BuildSSDIndex")
index.SetBuildParam("SearchInternalResultNum", "64", "BuildSSDIndex")

if (os.path.exists("spann_index")):
    shutil.rmtree("spann_index")

if index.BuildWithMetaData(x, m, vector_number, False, False):
    index.Save("spann_index") # Save the index to the disk

os.listdir('spann_index')

['HeadIndex',
 'indexloader.ini',
 'metadata.bin',
 'metadataIndex.bin',
 'SPTAGFullList.bin',
 'SPTAGHeadVectorIDs.bin',
 'SPTAGHeadVectors.bin']

In [20]:
index = SPTAG.AnnIndex.Load('spann_index')
result = index.SearchWithMetaData(q, 12) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

[26, 411, 990, 122, 207, 367, 983, 788, 869, 896, 108, 849]
[11.223834037780762, 11.411367416381836, 11.42112922668457, 11.483798027038574, 11.508404731750488, 11.629104614257812, 11.821345329284668, 11.8322114944458, 11.969610214233398, 11.978944778442383, 11.99281120300293, 12.137895584106445]
[b'26\n', b'411\n', b'990\n', b'122\n', b'207\n', b'367\n', b'983\n', b'788\n', b'869\n', b'896\n', b'108\n', b'849\n']


In [15]:
import SPTAG

index = SPTAG.AnnIndex('SPANN', 'UInt8', int(vector_dimension / 2))

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("IndexAlgoType", "BKT", "Base")
index.SetBuildParam("IndexDirectory", "spann_quan_index", "Base")
index.SetBuildParam("DistCalcMethod", "L2", "Base")
index.SetBuildParam("QuantizerFilePath", "quantizer.bin", "Base")

index.SetBuildParam("isExecute", "true", "SelectHead")
index.SetBuildParam("NumberOfThreads", "4", "SelectHead")
index.SetBuildParam("Ratio", "0.2", "SelectHead") # index.SetBuildParam("Count", "200", "SelectHead")

index.SetBuildParam("isExecute", "true", "BuildHead")
index.SetBuildParam("RefineIterations", "3", "BuildHead")
index.SetBuildParam("NumberOfThreads", "4", "BuildHead")

index.SetBuildParam("isExecute", "true", "BuildSSDIndex")
index.SetBuildParam("BuildSsdIndex", "true", "BuildSSDIndex")
index.SetBuildParam("PostingPageLimit", "12", "BuildSSDIndex")
index.SetBuildParam("SearchPostingPageLimit", "12", "BuildSSDIndex")
index.SetBuildParam("NumberOfThreads", "4", "BuildSSDIndex")
index.SetBuildParam("InternalResultNum", "32", "BuildSSDIndex")
index.SetBuildParam("SearchInternalResultNum", "64", "BuildSSDIndex")

if (os.path.exists("spann_quan_index")):
    shutil.rmtree("spann_quan_index")

if index.LoadQuantizer("quantizer.bin") and index.BuildWithMetaData(quan_x, m, vector_number, False, False):
    index.Save("spann_quan_index") # Save the index to the disk

os.listdir('spann_quan_index')

['HeadIndex',
 'indexloader.ini',
 'metadata.bin',
 'metadataIndex.bin',
 'quantizer.bin',
 'SPTAGFullList.bin',
 'SPTAGHeadVectorIDs.bin',
 'SPTAGHeadVectors.bin']

In [16]:
index = SPTAG.AnnIndex.Load('spann_quan_index')
index.SetQuantizerADC(True)
result = index.SearchWithMetaData(q, 12) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

[27, 671, 566, 905, 203, 277, 28, 178, 135, 845, 404, 54]
[10.690985679626465, 10.709616661071777, 10.901339530944824, 11.261568069458008, 11.372068405151367, 11.45018196105957, 11.7642183303833, 11.848787307739258, 12.02137565612793, 12.052336692810059, 12.079133033752441, 12.168403625488281]
[b'27\n', b'671\n', b'566\n', b'905\n', b'203\n', b'277\n', b'28\n', b'178\n', b'135\n', b'845\n', b'404\n', b'54\n']


## Online serve the index

Start the vector search service on the host machine which listens for the client requests on the port 8000

> Write a server configuration file **service.ini** as follows:

```bash
[Service]
ListenAddr=0.0.0.0
ListenPort=8000
ThreadNumber=8
SocketThreadNumber=8

[QueryConfig]
DefaultMaxResultNumber=6
DefaultSeparator=|

[Index]
List=MyIndex

[Index_MyIndex]
IndexFolder=sptag_index
```

> Start the server on the host machine

```bash
Server.exe -m socket -c service.ini
```

It will print the follow messages:

```bash
Setting TreeFilePath with value tree.bin
Setting GraphFilePath with value graph.bin
Setting VectorFilePath with value vectors.bin
Setting DeleteVectorFilePath with value deletes.bin
Setting BKTNumber with value 1
Setting BKTKmeansK with value 32
Setting BKTLeafSize with value 8
Setting Samples with value 1000
Setting TPTNumber with value 32
Setting TPTLeafSize with value 2000
Setting NumTopDimensionTpTreeSplit with value 5
Setting NeighborhoodSize with value 32
Setting GraphNeighborhoodScale with value 2
Setting GraphCEFScale with value 2
Setting RefineIterations with value 2
Setting CEF with value 1000
Setting MaxCheckForRefineGraph with value 8192
Setting NumberOfThreads with value 4
Setting DistCalcMethod with value Cosine
Setting DeletePercentageForRefine with value 0.400000
Setting AddCountForRebuild with value 1000
Setting MaxCheck with value 8192
Setting ThresholdOfNumberOfContinuousNoBetterPropagation with value 3
Setting NumberOfInitialDynamicPivots with value 50
Setting NumberOfOtherDynamicPivots with value 4
Load Vector From sptag_index\vectors.bin
Load Vector (100, 10) Finish!
Load BKT From sptag_index\tree.bin
Load BKT (1,101) Finish!
Load Graph From sptag_index\graph.bin
Load Graph (100, 32) Finish!
Load DeleteID From sptag_index\deletes.bin
Load DeleteID (100, 1) Finish!
Start to listen 0.0.0.0:8000 ...
```

> Start python client to connect to the server and send vector search request.

In [ ]:
import SPTAGClient
import time

# connect to the server
client = SPTAGClient.AnnClient('127.0.0.1', '8000')
while not client.IsConnected():
    time.sleep(1)
client.SetTimeoutMilliseconds(18000)

k = 3
vector_dimension = 10
# prepare query vector
q = np.random.rand(vector_dimension).astype(np.float32)

result = client.Search(q, k, 'Float', True) # AnnClient.Search(query_vector, knn, data_type, with_metadata)

print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

